## ***מחברת קוד לעיבוד נתוני תוצאות אלכם - סקר קרקע***

<bdi>

ברירת המחדל של הטבלה הינה הצגת חריגות מערך ה-VSL.

ניתן להציג חריגות מערך סף נוסף בהתאם למאפייני האתר.

בתא השני נבחר את התרכובות אותם נרצה להציג בטבלת התוצאות. לדוגמה: TPH, MTBE BTEX וכו'.

סימון האופציה **Add_exceptations** יציג בטבלה את כל תרכובות ה-VOC בהם יש חריגות מערכי הסף הנבחרים, VSL ו-TIER 1.

ב-**Add_specifics** נוסיף מספרי CAS של תרכובות נוספות אותן נרצה להציג על אף שלא אותרו בהן חריגות מערכי הסף.


על מנת לעשות זאת, יש לרשום במקום המיועד לכך לפי הפורמט הבא:

'cas number, cas number'

 לדוגמה: '106-93-4, 74-83-9'

 ניתן הוסיף כמה אנליזות שרוצים, 1,2,3 וכו'.

 אם תבוקש אנליזה שלא נמצאת בתוצאות מעבדה תקפוץ הודעה, יש ללחוץ אישור ולבדוק איזה אנליזה לא באמת קיימת בקובץ תוצאות שהעלתם

</bdi>

In [1]:
#@title נא לבחור פרמטרים מתאימים  { run: "auto" }

Tier1_type = "Tier 1 industrial A 0-6m" # @param ["Tier 1 Residential A 0-6m","Tier 1 Residential A >6m", "Tier 1 Residential B","Tier 1 industrial A 0-6m","Tier 1 industrial A >6m","Tier 1 industrial B"]

testsׁ_basic = "Metals" # @param ["TPH","MTBE BTEX","TPHּּּ + MTBE BTEX", "Metals", "TPH + Metals", "MTBE BTEX + Metals", "TPHּּּ + MTBE BTEX + Metals"]

Add_exceptations = False # @param {type:"boolean"}

Add_specifics = None # @param {type:"raw"}

file_name = 'תוצאות אשכול מתכות סופי' # @param {type:"string"}

Font_size_hebrew = 9 # @param {type:"slider", min:6, max:24, step:1}

Font_name_hebrew = 'David' # @param {type:"string"}

Font_size_english = 8 # @param {type:"slider", min:6, max:24, step:1}

Font_name_english = 'Times New Roman' # @param {type:"string"}

!pip install openpyxl
from openpyxl.styles import Font
import pandas as pd
import re
from IPython.display import Javascript

if Add_specifics == None:
  Add_specifics = ''


In [2]:
#@title להעלות קובץ ערכי סף של קרקע בלחיצה על הכפתור למטה
from google.colab import files
uploaded = files.upload()

filenames = list(uploaded.keys())

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through each file and read it based on its extension
for filename in filenames:
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
    elif filename.endswith('.xlsx'):
        df = pd.read_excel(filename)
    else:
        print(f"Unsupported file format: {filename}")
        continue
        # Round all numeric cells to 2 decimal points
    #df = df.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)

    dataframes.append(df)

# Concatenate all dataframes into one
if dataframes:
    threshold_data = pd.concat(dataframes, ignore_index=True)
else:
    print("No valid files uploaded.")


Saving גרסה 7 דצמבר 2024 - soil_threshold.csv to גרסה 7 דצמבר 2024 - soil_threshold (3).csv


In [18]:
#@title להעלות קובץ טבלת תוצאות של קרקע בלחיצה על הכפתור למטה

from google.colab import files
import pandas as pd

# Upload files
uploaded = files.upload()

# List of uploaded filenames
filenames = list(uploaded.keys())

# Initialize empty lists to store dataframes for TPH, IPC, and VOC splits
dataframes_TPH = []
dataframes_ICP = []
dataframes_VOC_splits = []

# Loop through each file and process based on extension
for filename in filenames:
    if filename.endswith('.xlsx'):
        # Read the TPH sheet
        try:
            df_TPH = pd.read_excel(filename, sheet_name='TPH')
            # Round numeric cells to 2 decimal points
            #df_TPH = df_TPH.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)
            dataframes_TPH.append(df_TPH)
        except Exception as e:
            print(f"Error reading TPH sheet of {filename}: {e}")

        # Read the IPC sheet if it exists
        try:
            df_ICP = pd.read_excel(filename, sheet_name='ICP', header=None)
            # Append the DataFrame to IPC list
            dataframes_ICP.append(df_ICP)
        except Exception as e:
            print(f"Error reading ICP sheet of {filename}: {e}")

        # Read the VOC sheet and process "Analysis Location:" splitting
        try:
            # Read the VOC sheet without treating the first row as headers
            df_VOC = pd.read_excel(filename, sheet_name='VOC', header=None)

            # Identify rows containing "Analysis Location:"
            analysis_location_rows = df_VOC[df_VOC.apply(
                lambda row: row.astype(str).str.contains("Analysis Location:").any(), axis=1)].index.tolist()

            # Count occurrences and split based on indices
            if len(analysis_location_rows) > 0:
                for i in range(len(analysis_location_rows)):
                    start_row = analysis_location_rows[i]
                    end_row = analysis_location_rows[i + 1] if i + 1 < len(analysis_location_rows) else None
                    split_df = df_VOC.iloc[start_row:end_row]
                    dataframes_VOC_splits.append(split_df.reset_index(drop=True))
            else:
                # If no "Analysis Location:", treat the entire sheet as one DataFrame
                dataframes_VOC_splits.append(df_VOC.reset_index(drop=True))

        except Exception as e:
            print(f"Error reading VOC sheet of {filename}: {e}")
    else:
        print(f"Unsupported file format: {filename}")

# Concatenate all TPH sheet dataframes into one
if dataframes_TPH:
    results_data_TPH = pd.concat(dataframes_TPH, ignore_index=True)
else:
    print("No valid TPH sheets found.")
    results_data_TPH = pd.DataFrame()

# Concatenate all IPC sheet dataframes into one
if dataframes_ICP:
    results_data_ICP = pd.concat(dataframes_ICP, axis=1)
else:
    print("No valid IPC sheets found.")
    results_data_ICP = pd.DataFrame()

# Concatenate all VOC splits horizontally, dropping the first 5 columns from the 2nd DataFrame onward
if dataframes_VOC_splits:
    # Keep the first 5 columns only from the first DataFrame
    processed_dfs = [dataframes_VOC_splits[0]]
    processed_dfs += [df.iloc[:, 5:].reset_index(drop=True) for df in dataframes_VOC_splits[1:]]

    # Align and concatenate by rows
    results_data_VOC_horizontal = pd.concat(processed_dfs, axis=1)
else:
    print("No valid VOC splits found.")
    results_data_VOC_horizontal = pd.DataFrame()

results_data_VOC = results_data_VOC_horizontal

# Convert the data to string to avoid errors
results_data_TPH = results_data_TPH.astype(str)
results_data_ICP = results_data_ICP.astype(str)
results_data_VOC = results_data_VOC.astype(str)

Saving table737449B-980.xlsx to table737449B-980 (17).xlsx
Saving table737452B-80.xlsx to table737452B-80 (17).xlsx
Saving table737485B-238.xlsx to table737485B-238 (17).xlsx
Saving table737497B-174.xlsx to table737497B-174 (17).xlsx
Saving table737549B-605.xlsx to table737549B-605 (12).xlsx
Saving עותק של table737550B-535.xlsx to עותק של table737550B-535 (21).xlsx


In [22]:
#@title יש להריץ את התא ולהוריד את הטבלה המוכנה

# Function to remove '(' and ')' from strings
def remove_parentheses(cell):
    if isinstance(cell, str):  # Only apply to strings
        return cell.replace('(', '').replace(')', '')
    return cell

# Apply to the entire dataframe for results_data_TPH
results_data_TPH = results_data_TPH.applymap(remove_parentheses)

# Apply to the entire dataframe for results_data_VOC
results_data_VOC = results_data_VOC.applymap(remove_parentheses)

# Apply to the entire dataframe for results_data_ICP
results_data_ICP = results_data_ICP.applymap(remove_parentheses)

drills_list = results_data_TPH["Analyte"].dropna().tolist()
drills_list = drills_list[1:]

# Function to replace "N.D." values
def replace_nd(cell, column):
    if cell == "N.D.":
        return "<15" if column == "Total TPH" else "<7"
    return cell

# Apply the replacement logic to the dataframe
results_data_TPH = results_data_TPH.apply(
    lambda col: col.apply(lambda cell: replace_nd(cell, col.name))
)

voc_analysis_row = results_data_VOC[results_data_VOC.apply(
    lambda row: row.astype(str).str.contains("Analysis Location:").any(), axis=1)]

if not voc_analysis_row.empty:
    # Extract the row values as a list
    voc_drills_list = voc_analysis_row.iloc[0].dropna().tolist()  # Drop NaN values

    voc_drills_list = voc_drills_list[5:]

    for voc_item in voc_drills_list:
        # Check if voc_item is in any item of drills_list
        if not any(voc_item in drill_item for drill_item in drills_list):
            drills_list.append(voc_item)  # Add to drills_list if no match is found

icp_analysis_row = results_data_ICP[results_data_ICP.apply(
    lambda row: row.astype(str).str.contains("Sample Name").any(), axis=1)]

if not icp_analysis_row.empty:
    if testsׁ_basic == "Metals":
      drills_list = []

    # Extract the row values as a list
    icp_drills_list = icp_analysis_row.iloc[0].dropna().tolist()  # Drop NaN values

    icp_drills_list = icp_drills_list[5:]

    for icp_item in icp_drills_list:

        # Check if voc_item is in any item of drills_list
        if not any(icp_item in drill_item for drill_item in drills_list) and "Metals" in testsׁ_basic:
            drills_list.append(icp_item)  # Add to drills_list if no match is found

# Define the structure of the DataFrame
columns = ['name', 'date', 'Unnamed: 2', 'depth', ' PID','full name']
data = [
    ['units', None, None, 'm', 'ppm', None],
    ['Cas', None, None, None, None, None],
    ['VSL', None, None, None, None, None],
    ['TIER 1', None, None, None, None, None]
]

# # Normalize 'dup', 'duplicate', or 'Duplicate' to 'DUP'
# normalized_drills_list = []
# for item in drills_list:
#     # Replace 'dup', 'duplicate', or 'Duplicate' with 'DUP' (case-insensitive)
#     normalized_item = re.sub(r'\b(?:dup|duplicate|Duplicate)\b', 'DUP', item, flags=re.IGNORECASE)
#     normalized_drills_list.append(normalized_item)

# # Remove duplicates by converting to a set, then back to a list to maintain unique items
# normalized_drills_list = list(set(normalized_drills_list))

# # Sort the list if you want to keep it in a specific order
# normalized_drills_list.sort()
# drills_list = normalized_drills_list

# Create the DataFrame
df_template = pd.DataFrame(data, columns=columns)

# # Process each item in drills_list
# df_template['full name'] = None
# for item in drills_list:
#     # Find the first instance of a number with a dot and numbers on both sides
#     match = re.search(r'(\d+\.\d+)', item)
#     if match:
#         # Extract the depth (value) and the name (key)
#         depth = match.group(1)  # The matched number with a dot
#         name = item[:match.start()-1].strip()  # Get everything before the depth
#         new_row = {'name': name, 'depth': depth, 'full name' : item}  # Create a new row with name and depth
#         df_template = pd.concat([df_template, pd.DataFrame([new_row])], ignore_index=True)
# # Fill missing columns with None to match the template structure
# df_template = df_template.reindex(columns=columns, fill_value=None)

# # Create the DataFrame
# df_template = pd.DataFrame(data, columns=columns)


# Process each item in drills_list
df_template['full name'] = None

# Updated extraction logic for name, depth, and type
def extract_name_and_depth(item):
    if not isinstance(item, str):
        return item, None, None

    # Detect flags
    is_dup = bool(re.search(r'\b(?:dup|duplicate|DUP|D)\b', item, re.IGNORECASE))
    is_split = bool(re.search(r'\b(?:split|SP)\b', item, re.IGNORECASE))

    # Clean flags from item for name extraction
    clean_item = re.sub(r'\b(?:dup|duplicate|DUP|D|split|SP)\b', '', item, flags=re.IGNORECASE).strip()

    # Match name and depth
    match_paren = re.search(r'(.+?)\s*\((\d+\.\d+)\)', clean_item)
    if match_paren:
        name = match_paren.group(1).strip()
        depth = match_paren.group(2)
    else:
        match_dash = re.search(r'^(.+?-\d+\.\d+)\s*-\s*(\d+\.\d+)', clean_item)
        if match_dash:
            name = match_dash.group(1).strip()
            depth = match_dash.group(2)
        else:
            match = re.findall(r'(\d+\.\d+)', clean_item)
            if match:
                depth = match[-1]
                name = clean_item[:clean_item.rfind(depth)].strip(" -")
            else:
                name, depth = clean_item.strip(), None

    return name, depth, ('**' if is_split else '*') if is_dup or is_split else ''

# Build a temp list to group regular, DUP, and split rows
temp_rows = {}

for item in drills_list:
    name, depth, suffix = extract_name_and_depth(item)
    if depth is None:
        continue
    key = (name, depth)

    # Initialize group
    if key not in temp_rows:
        temp_rows[key] = []

    # Compose adjusted depth
    depth_display = f"{depth}{suffix}"
    temp_rows[key].append({
        'name': name,
        'depth': depth_display,
        'full name': item
    })

# Flatten into df_template in the order: original, DUP, split
for key in sorted(temp_rows.keys()):
    # Ensure order: normal -> * -> **
    ordered = sorted(temp_rows[key], key=lambda x: len(x['depth']))
    for row_data in ordered:
        new_row = {**row_data}
        df_template = pd.concat([df_template, pd.DataFrame([new_row])], ignore_index=True)

# Ensure columns match
df_template = df_template.reindex(columns=columns, fill_value=None)


# Define the starting row for sorting
start_row = 4

# Extract the header rows (rows before start_row) and the rows to be sorted
header_rows = df_template.iloc[:start_row]
rows_to_sort = df_template.iloc[start_row:]

# Extract base name (e.g., ק) and numeric part as float (e.g., 1.1 from ק-1.1)
rows_to_sort['name_prefix'] = rows_to_sort['name'].str.extract(r'([^-\d]+)', expand=False)
rows_to_sort['name_number'] = rows_to_sort['name'].str.extract(r'(\d+\.\d+|\d+)', expand=False).astype(float)

# Sort by prefix, then by numeric name, then by depth
sorted_rows = rows_to_sort.sort_values(
    by=['name_prefix', 'name_number', 'depth'],
    key=lambda col: pd.to_numeric(col, errors='coerce') if col.name in ['name_number', 'depth'] else col
)

# Drop the temporary columns used for sorting
sorted_rows = sorted_rows.drop(columns=['name_prefix', 'name_number'])

# Concatenate the header rows with the sorted rows
df_sorted = pd.concat([header_rows, sorted_rows], ignore_index=True)

#####סידור תוצאות נדיפים######
results_data_VOC_op = results_data_VOC.T

# Replace "<MDL" and "<MRL" based on values in rows 2 and 3
def replace_mdl_mrl(cell, column):
    if isinstance(cell, str) and "<MDL" in cell:
        return f"<{results_data_VOC_op.iloc[2, column]}"
    elif isinstance(cell, str) and "<MRL" in cell:
        return f"<{results_data_VOC_op.iloc[3, column]}"
    return cell

# Iterate over each column and apply the replacement
for col in results_data_VOC_op.columns[2:]:  # Skip first two columns which are metadata
    results_data_VOC_op[col] = results_data_VOC_op[col].apply(lambda x: replace_mdl_mrl(x, results_data_VOC_op.columns.get_loc(col)))

# Remove rows 0, 2, 3, and 4 and column 1
rows_to_remove = [0, 2, 3, 4]
results_data_VOC_op = results_data_VOC_op.drop(index=rows_to_remove).reset_index(drop=True)
results_data_VOC_op = results_data_VOC_op.drop(columns=results_data_VOC_op.columns[1])

#####סידור תוצאות מתכות######
if not results_data_ICP.empty:
  results_data_ICP_op = results_data_ICP.T

  # Replace "N.D." and "<LOQ" based on values in rows 2 and 3
  def replace_nd_loq(cell, column):
      if isinstance(cell, str) and "N.D." in cell:
          return f"<{results_data_ICP_op.iloc[1, column]}"
      elif isinstance(cell, str) and "<LOQ" in cell:
          return f"<{results_data_ICP_op.iloc[2, column]}"
      return cell

  # Iterate over each column and apply the replacement
  for col in results_data_ICP_op.columns[2:]:  # Skip first two columns which are metadata
      results_data_ICP_op[col] = results_data_ICP_op[col].apply(lambda x: replace_nd_loq(x, results_data_ICP_op.columns.get_loc(col)))

  # Remove rows 0, 2, 3, and 4 and column 1
  rows_to_remove = [1, 2, 3]
  results_data_ICP_op = results_data_ICP_op.drop(index=rows_to_remove).reset_index(drop=True)
  results_data_ICP_op = results_data_ICP_op.drop(columns=results_data_ICP_op.columns[1])

  metals_dic = {'Ag' : '7440-22-4', 'Al':'7429-90-5' , 'As' : '7440-38-2',
  'Ba' : '7440-39-3', 'Be' : '7440-41-7', 'Cd' : '7440-43-9',
  'Co' : '7440-48-4', 'Cr' : '7440-47-3', 'Cu' : '7440-50-8',
  'Fe' : '7439-89-6', 'Hg' : '7439-97-6', 'Li' : '7439-93-2',
  'Mn' : '7439-96-5', 'Mo' : '7439-98-7', 'Ni' : '7440-02-0',
  'Pb' : '7439-92-1', 'Sb' : '7440-36-0', 'Se' : '7782-49-2',
  'Tl' : '7440-28-0', 'V' : '7440-62-2', 'Zn' : '7440-66-6'}

  # Replace row 0 values in results_data_ICP_op based on metals_dic
  results_data_ICP_op.iloc[0] = results_data_ICP_op.iloc[0].map(metals_dic).fillna(results_data_ICP_op.iloc[0])

  results_data_VOC_op.iloc[0, 0] = 'Sample Name'
  result_outer = pd.merge(results_data_VOC_op, results_data_ICP_op, on=0, how='outer')

  # Define the custom order based on sorted_rows['full name']
  custom_order = sorted_rows['full name']

  # Ensure column 0 in result_outer is treated as a categorical variable with the custom order
  result_outer[0] = pd.Categorical(result_outer[0], categories=custom_order, ordered=True)

  # Sort the DataFrame by column 0
  result_outer_sorted = result_outer.sort_values(by=0)

  # Reset the index if needed
  result_outer_sorted.reset_index(drop=True, inplace=True)
  result_outer_sorted = pd.concat([result_outer_sorted.iloc[[-1]], result_outer_sorted.iloc[:-1]]).reset_index(drop=True)

  results_data_ICP_op = results_data_ICP_op.replace("Sample Name", "")
  results_data_VOC_op = result_outer_sorted
  results_data_VOC_op.columns = range(len(results_data_VOC_op.columns))

else:
  print('ICP sheet is empty')

########הכנסת נתונים לטמפלייט########
if 'TPH' in testsׁ_basic:
  df_sorted['TPH DRO'] = None
  df_sorted['TPH ORO'] = None
  df_sorted['Total TP'] = None

  # Ensure column names are consistent and merge based on "full name" and "Analyte"
  merged_df = df_sorted.merge(
      results_data_TPH.rename(columns={"Analyte": "full name"}),
      on="full name",
      how="left"
  )

  # Update the relevant columns in df_sorted with data from results_data_TPH
  df_sorted['TPH DRO'] = merged_df['DRO']
  df_sorted['TPH ORO'] = merged_df['ORO']
  df_sorted['Total TP'] = merged_df['Total TPH']

  # Renaming the column 'Total TP' to 'Total TPH'
  df_sorted.rename(columns={'Total TP': 'Total TPH'}, inplace=True)

  #Adding CAS number
  df_sorted['TPH DRO'][1] = "C10-C40"
  df_sorted['TPH ORO'][1] = "C10-C40"
  df_sorted['Total TPH'][1] = "C10-C40"

if 'MTBE' in testsׁ_basic:
  df_sorted['MTBE'] = None
  df_sorted['Benzene'] = None
  df_sorted['Toluene'] = None
  df_sorted['Ethylbenzene'] = None
  df_sorted['Xylene'] = None

  #Adding CAS number
  df_sorted['MTBE'][1] = '1634-04-4'
  df_sorted['Benzene'][1] = '71-43-2'
  df_sorted['Toluene'][1] = '108-88-3'
  df_sorted['Ethylbenzene'][1] = '100-41-4'
  df_sorted['Xylene'][1] = '95-47-6'



  # Ensure matching names are correctly formatted
  df_sorted['full name'] = df_sorted['full name'].astype(str).str.strip()
  results_data_VOC_op[0] = results_data_VOC_op[0].astype(str).str.strip()

  # Extract CAS mappings for the required columns
  cas_map = df_sorted.loc[1, ["MTBE", "Benzene", "Toluene", "Ethylbenzene", "Xylene"]].to_dict()

  # Iterate over each row in df_sorted and fill in the values
  for index, row in df_sorted.iterrows():
      if index <= 1:  # Skip non-data rows
          continue
      name = row['full name']
      if pd.isna(name):
          continue

      # Find the matching name in results_data_VOC_op (reversed logic)
      matching_row = results_data_VOC_op[results_data_VOC_op[0].apply(lambda x: x == name if pd.notna(x) else False)]
      if matching_row.empty:
          continue

      for column, cas in cas_map.items():
          if pd.isna(cas):
              continue

          # Find the column in results_data_VOC_op that matches the CAS
          try:
              cas_column_index = results_data_VOC_op.iloc[0, :].tolist().index(cas)  # Exact CAS match in the first row
              value_to_fill = matching_row.iloc[0, cas_column_index]  # Extract the correct value
              df_sorted.at[index, column] = value_to_fill
          except ValueError:
              # CAS not found in the first row
              continue
# Adding metals to df_sorted if 'Metals' is in tests_basic
if 'Metals' in testsׁ_basic:
    # Initialize columns for each metal in df_sorted
    for metal, cas in metals_dic.items():
        df_sorted[metal] = None

    # Add CAS numbers in the second row (index 1) for each metal
    for metal, cas in metals_dic.items():
        df_sorted.at[1, metal] = cas

    # Ensure matching names are correctly formatted
    df_sorted['full name'] = df_sorted['full name'].astype(str).str.strip()
    results_data_ICP_op[0] = results_data_ICP_op[0].astype(str).str.strip()

    # Iterate over each row in df_sorted and fill in the values
    for index, row in df_sorted.iterrows():
        if index <= 1:  # Skip non-data rows
            continue
        name = row['full name']
        if pd.isna(name):
            continue

        # Find the matching name in results_data_ICP_op
        matching_row = results_data_ICP_op[results_data_ICP_op[0].apply(lambda x: x == name if pd.notna(x) else False)]
        if matching_row.empty:
            continue

        # Iterate over metals and fill their values from results_data_ICP_op
        for metal, cas in metals_dic.items():
            try:
                # Find the column in results_data_ICP_op that matches the CAS
                cas_column_index = results_data_ICP_op.iloc[0, :].tolist().index(cas)  # Exact CAS match in the first row
                value_to_fill = matching_row.iloc[0, cas_column_index]  # Extract the correct value
                df_sorted.at[index, metal] = value_to_fill
            except ValueError:
                # CAS not found in the first row
                continue

if Add_exceptations == True:
    # Ensure CAS numbers in both dataframes are properly formatted
    results_data_VOC_op[0] = results_data_VOC_op[0].astype(str).str.strip()
    threshold_data['CAS No.'] = threshold_data['CAS No.'].astype(str).str.strip()

    # Initialize a new row with NaN values
    new_row = [None] * results_data_VOC_op.shape[1]

    # Iterate through each CAS number in the first row of results_data_VOC_op
    for col_index, cas_number in enumerate(results_data_VOC_op.iloc[0, :]):
        if pd.isna(cas_number) or cas_number == 'nan':
            continue

        # Find the matching row in threshold_data
        matching_row = threshold_data[threshold_data['CAS No.'] == cas_number]
        if not matching_row.empty:
            # Get the value from the "VSL" column
            vsl_value = matching_row.iloc[0]['VSL']
            new_row[col_index] = vsl_value

    # Append the new row to results_data_VOC_op
    results_data_VOC_op.loc[len(results_data_VOC_op)] = new_row

    ## Adding the values
    # Iterate through each column in results_data_VOC_op
    for col_index in range(results_data_VOC_op.shape[1]):
        cas_number = results_data_VOC_op.iloc[0, col_index]  # Get the CAS number from row 0
        if pd.isna(cas_number) or cas_number == 'nan':
            continue

        # Check if the CAS number is already in df_sorted
        if cas_number in df_sorted.iloc[1, :].values:
            continue  # Skip if the CAS number is already present

        # Get the column values excluding the first row (CAS number)
        column_values = results_data_VOC_op.iloc[1:, col_index]

        # Skip columns where all values are NaN
        if column_values.isna().all():
            continue

        # Filter column values to keep only numeric ones
        numeric_values = pd.to_numeric(column_values, errors='coerce')

        # Get the VSL value (assuming it's in the last row of the column)
        vsl_value = results_data_VOC_op.iloc[-1, col_index]
        try:
            vsl_value = float(vsl_value)  # Ensure VSL value is a float
        except ValueError:
            continue  # Skip columns where VSL is not a valid number

        # **Check for numeric values exceeding VSL**
        if (numeric_values > vsl_value).any():
            matching_row = threshold_data[threshold_data['CAS No.'] == cas_number]
            header_name = matching_row.iloc[0, -1] if not matching_row.empty else f"Column_{cas_number}"

            # Add new column to df_sorted
            df_sorted[header_name] = None
            df_sorted.at[1, header_name] = cas_number

            # Fill values where 'full name' matches
            for index, row in df_sorted.iterrows():
                if index <= 1:
                    continue
                name = row['full name']
                if pd.isna(name):
                    continue

                matching_row = results_data_VOC_op[
                    results_data_VOC_op[0].apply(lambda x: x == name if pd.notna(x) else False)
                ]
                if matching_row.empty:
                    continue

                value_to_fill = matching_row.iloc[0, col_index]
                df_sorted.at[index, header_name] = value_to_fill

        # **Check for "<..." values where the extracted number is greater than VSL**
        for cell_value in column_values:
            if isinstance(cell_value, str) and cell_value.startswith("<"):
                match = re.search(r'<\s*(\d+(\.\d+)?)', cell_value)
                if match:
                    try:
                        extracted_value = float(match.group(1))  # Extracted number from "<..."

                        if extracted_value > vsl_value:  # Compare with VSL
                            # Add column if condition is met
                            matching_row = threshold_data[threshold_data['CAS No.'] == cas_number]
                            header_name = (
                                matching_row.iloc[0, -1]
                                if not matching_row.empty
                                else f"Column_{cas_number}_BDL"
                            )

                            # Ensure column exists in df_sorted
                            if header_name not in df_sorted.columns:
                                df_sorted[header_name] = None
                                df_sorted.at[1, header_name] = cas_number

                            # Fill values where 'full name' matches
                            for index, row in df_sorted.iterrows():
                                if index <= 1:
                                    continue
                                name = row['full name']
                                if pd.isna(name):
                                    continue

                                matching_row = results_data_VOC_op[
                                    results_data_VOC_op[0].apply(lambda x: x == name if pd.notna(x) else False)
                                ]
                                if matching_row.empty:
                                    continue

                                value_to_fill = matching_row.iloc[0, col_index]
                                df_sorted.at[index, header_name] = value_to_fill

                    except ValueError:
                        pass  # Skip invalid values

if Add_specifics:
    Add_specifics_list = [item.strip() for item in Add_specifics.split(',')]
    for item in Add_specifics_list:
        if item not in results_data_VOC_op.iloc[0].values:
            display(Javascript('alert("⚠️ one or more of the specific CAS numbers you insert is not in lab results ⚠️")'))
            print(f'{item} is not in lab results')
        else:
            # Find the column index of the item in results_data_VOC_op
            try:
                col_index = results_data_VOC_op.iloc[0, :].tolist().index(item)
            except ValueError:
                print(f'{item} not found in results_data_VOC_op header.')
                continue

            # Check if the CAS number (item) is already in df_sorted
            if item in df_sorted.iloc[1, :].values:
                continue  # Skip if the CAS number is already present

            # Find the row in results_data_VOC where the CAS number matches
            matching_row = threshold_data[
                threshold_data['CAS No.'] == item  # Match CAS number in column 1
            ]

            # If a match is found, get the header from column 0 in the matching row
            if not matching_row.empty:
                header_name = matching_row.iloc[0, -1]
            else:
                header_name = f"Column_{item}"  # Fallback to a generic name

            # Add a new column to df_sorted with the header_name
            df_sorted[header_name] = None
            df_sorted.at[1, header_name] = item

            # Add matching values to df_sorted for the rows where 'full name' matches
            for index, row in df_sorted.iterrows():
                if index <= 1:  # Skip the header and CAS rows
                    continue
                name = row['full name']
                if pd.isna(name):
                    continue

                # Find the matching row in results_data_VOC_op
                matching_row = results_data_VOC_op[
                    results_data_VOC_op[0].apply(lambda x: x == name if pd.notna(x) else False)
                ]
                if matching_row.empty:
                    continue

                # Extract the value from results_data_VOC_op for the current column
                value_to_fill = matching_row.iloc[0, col_index]
                df_sorted.at[index, header_name] = value_to_fill


df_sorted.iloc[0, 6:] = 'mg/kg'

# Iterate through each column in df_sorted starting from column 6 onward
for col_index in range(6, len(df_sorted.columns)):
    cas_number = df_sorted.iat[1, col_index]  # Get the CAS number from row 1

    if pd.isna(cas_number) or cas_number == '':  # Skip if CAS number is empty or NaN
        continue

    # Find the matching row in threshold_data based on CAS No.
    matching_row = threshold_data[threshold_data['CAS No.'] == cas_number]

    if not matching_row.empty:
        # Extract the VSL value from the matching row
        vsl_value = matching_row.iloc[0]['VSL']
        # Populate the cell in row 2 of df_sorted for the current column
        df_sorted.iat[2, col_index] = vsl_value

        # Extract the value from the column with the name Tier1_type
        if Tier1_type in matching_row.columns:
            tier1_value = matching_row.iloc[0][Tier1_type]
            # Populate the cell in row 3 of df_sorted for the current column
            df_sorted.iat[3, col_index] = tier1_value

df_sorted = df_sorted.drop(columns='full name')

########סידור לטבלת אקסל להורדה########

# Fill empty values from row 5 and column 6 onward with "-"
for row_idx in range(4, len(df_sorted)):
    for col_idx in range(5, len(df_sorted.columns)):
        val = df_sorted.iat[row_idx, col_idx]
        if pd.isna(val) or val == '':
            df_sorted.iat[row_idx, col_idx] = "-"

# Fill empty values in rows 3 and 4 (index 2 and 3) and column 6 onward with "--"
for row_idx in [2, 3]:
    for col_idx in range(5, len(df_sorted.columns)):
        val = df_sorted.iat[row_idx, col_idx]
        if pd.isna(val) or val == '':
            df_sorted.iat[row_idx, col_idx] = "--"

from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import re

# Create an Excel workbook and add a worksheet
wb = Workbook()
ws = wb.active

# Set the sheet to right-to-left
ws.sheet_view.rightToLeft = True

thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

alignment_center_wrap = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Define colors for conditional formatting
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
custom_fill = PatternFill(start_color="F7C7AC", end_color="F7C7AC", fill_type="solid")
gray_fill = PatternFill(start_color="D3D3D3", end_color="D3D3D3", fill_type="solid")

# Step 1: Add the merged cells for the headers
ws.merge_cells(start_row=1, start_column=1, end_row=5, end_column=1)
cell = ws.cell(row=1, column=1)
cell.value = "שם קידוח"
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

for i in range(2, 6):
  cell = ws.cell(row=i, column=1)
  cell.border = Border(
      left=Side(style='thin'),
      right=Side(style='thin'))

for i in range(2, 6):  # Iterate over columns (2 to 5 inclusive)
    for j in range(1, len(df_sorted)+2):  # Iterate over rows (1 to len(df_sorted) - 1)
        cell = ws.cell(row=j, column=i)
        cell.border = Border(bottom=Side(style='thin'))

ws.merge_cells(start_row=1, start_column=2, end_row=5, end_column=2)
cell = ws.cell(row=1, column=2)
cell.value = "תאריך"
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=1, column=3)
cell.value = ""
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=2, column=3)
cell.value = "יחידות"
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=1, column=4)
cell.value = "עומק"
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=2, column=4)
cell.value = "m"
cell.font = Font(name=Font_name_english, size=Font_size_english)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=1, column=5)
cell.value = "PID"
cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=2, column=5)
cell.value = "ppm"
cell.font = Font(name=Font_name_english, size=Font_size_english)
cell.alignment = alignment_center_wrap
cell.border = thin_border

ws.merge_cells(start_row=3, start_column=3, end_row=3, end_column=5)
cell = ws.cell(row=3, column=3)
cell.value = "CAS"
cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

ws.merge_cells(start_row=4, start_column=3, end_row=4, end_column=5)
cell = ws.cell(row=4, column=3)
cell.value = "VSL"
cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

ws.merge_cells(start_row=5, start_column=3, end_row=5, end_column=5)
cell = ws.cell(row=5, column=3)
cell.value = "TIER 1"
cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

# Helper function to detect if a string contains English letters
def contains_english(value):
    return any(char.isascii() and char.isalpha() for char in str(value))

# Step 2: Add the 'name' column to the first column in the Excel file starting from row 4 of df_sorted
for row_idx, value in enumerate(df_sorted["name"][4:], start=6):
    cell = ws.cell(row=row_idx, column=1)
    cell.value = value
    cell.font = Font(
        name=Font_name_english if contains_english(value) else Font_name_hebrew,
        size=Font_size_english if contains_english(value) else Font_size_hebrew,
    )
    cell.alignment = alignment_center_wrap
    cell.border = thin_border

# Merge cells in column 1 with the same value
current_start = 6
previous_value = None
for row_idx, value in enumerate(df_sorted["name"][4:], start=6):
    if value == previous_value:
        continue
    if previous_value is not None:
        ws.merge_cells(start_row=current_start, start_column=1, end_row=row_idx - 1, end_column=1)
    current_start = row_idx
    previous_value = value
# Merge the last group
if previous_value is not None:
    ws.merge_cells(start_row=current_start, start_column=1, end_row=len(df_sorted["name"][4:]) + 5, end_column=1)

# Step 3: Add the 'depth' column to column 4 in the Excel file starting from row 6
for row_idx, value in enumerate(df_sorted["depth"][4:], start=6):
    cell = ws.cell(row=row_idx, column=4)
    cell.value = value
    cell.font = Font(
        name=Font_name_english if contains_english(value) else Font_name_hebrew,
        size=Font_size_english if contains_english(value) else Font_size_hebrew,
    )
    cell.alignment = alignment_center_wrap
    cell.border = thin_border

# Step 4: Add data from column 6 onward starting from row 1 in Excel
for col_idx, col_name in enumerate(df_sorted.columns[5:], start=6):  # Iterate through columns 6 onward
    # Add column header
    header_cell = ws.cell(row=1, column=col_idx)
    header_cell.value = col_name
    header_cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
    header_cell.alignment = alignment_center_wrap
    header_cell.border = thin_border

    # Add column data and round float values
    for row_idx, value in enumerate(df_sorted[col_name], start=2):  # Start from row 2 in Excel
        cell = ws.cell(row=row_idx, column=col_idx)
        try:
            value_numeric = round(float(value), 4)  # Round to 4 decimal places
            cell.value = value_numeric
        except (ValueError, TypeError):
            cell.value = value  # Non-numeric values remain unchanged

        cell.font = Font(
            name=Font_name_english if contains_english(value) else Font_name_hebrew,
            size=Font_size_english if contains_english(value) else Font_size_hebrew,
        )
        cell.alignment = alignment_center_wrap
        cell.border = thin_border

        # Conditional formatting for rows 6 and onward
        if row_idx >= 6:
            try:
                row_4_value = float(ws.cell(row=4, column=col_idx).value)
                row_5_value = float(ws.cell(row=5, column=col_idx).value)

                if isinstance(cell.value, (float, int)):
                    if cell.value > row_4_value:
                        cell.fill = yellow_fill
                        cell.font = Font(bold=True, name=cell.font.name, size=cell.font.size)
                    if cell.value > row_5_value:
                        cell.fill = custom_fill
                        cell.font = Font(bold=True, name=cell.font.name, size=cell.font.size)

                # **Gray Fill Logic for "<..." values (Handles both Integers & Floats)**
                elif isinstance(cell.value, str) and cell.value.startswith("<"):
                    match = re.search(r'<\s*(\d+(\.\d+)?)', cell.value)  # Extract numeric part

                    if match:
                        try:
                            numeric_value = float(match.group(1))  # Convert extracted value to float
                            threshold_value = ws.cell(row=4, column=col_idx).value  # Get row 4 value

                            # Ensure threshold value is numeric before comparison
                            if isinstance(threshold_value, (int, float)) and numeric_value > threshold_value:
                                cell.fill = gray_fill
                                cell.font = Font(bold=True, name=cell.font.name, size=cell.font.size)
                        except ValueError:
                            pass  # Skip if conversion fails

            except (ValueError, TypeError):
                pass  # Skip cells that cannot be compared numerically



# Merge columns 2 and 3 for each row from row 6 onward
for row_idx in range(6, len(df_sorted["name"]) + 6):
    ws.merge_cells(start_row=row_idx, start_column=2, end_row=row_idx, end_column=3)

# Define the medium border
medium_side = Side(style="medium")

# Define the range for the table
start_row = 1  # Start from the first row
end_row = len(df_sorted)+1  # Dynamically determine the last row based on the data
start_col = 1  # Start from the first column
end_col = ws.max_column  # Dynamically determine the last column based on the data

# Apply medium border to the outer edges of the table
for row in range(start_row, end_row + 1):
    for col in range(start_col, end_col + 1):
        cell = ws.cell(row=row, column=col)

        # Apply borders to the outer edges only
        if row == start_row:  # Top edge
            cell.border = Border(top=medium_side, left=cell.border.left, right=cell.border.right, bottom=cell.border.bottom)
        if row == end_row:  # Bottom edge
            cell.border = Border(bottom=medium_side, left=cell.border.left, right=cell.border.right, top=cell.border.top)
        if col == start_col:  # Left edge
            cell.border = Border(left=medium_side, top=cell.border.top, bottom=cell.border.bottom, right=cell.border.right)
        if col == end_col:  # Right edge
            cell.border = Border(right=medium_side, top=cell.border.top, bottom=cell.border.bottom, left=cell.border.left)

# Define the range for the first 5 rows
start_row = 1
end_row = 5
start_col = 1
end_col = ws.max_column  # Dynamically determine the last column based on the data

# Apply thick border to the outer edges
for row in range(start_row, end_row + 1):
    for col in range(start_col, end_col + 1):
        cell = ws.cell(row=row, column=col)

        # Apply borders to the outer edges only
        if row == start_row:  # Top edge
            cell.border = Border(top=medium_side, left=cell.border.left, right=cell.border.right, bottom=cell.border.bottom)
        if row == end_row:  # Bottom edge
            cell.border = Border(bottom=medium_side, left=cell.border.left, right=cell.border.right, top=cell.border.top)
        if col == start_col:  # Left edge
            cell.border = Border(left=medium_side, top=cell.border.top, bottom=cell.border.bottom, right=cell.border.right)
        if col == end_col:  # Right edge
            cell.border = Border(right=medium_side, top=cell.border.top, bottom=cell.border.bottom, left=cell.border.left)

# Determine the last row and column of the table
last_row = ws.max_row
start_col = 1  # Assuming you want the legend to start from the first column

# Add legend for DUP and Split notation
legend_dup_split = ws.cell(row=len(df_sorted)+2, column=start_col)
legend_dup_split.value = "*Duplicate | **Split"
legend_dup_split.font = Font(bold=True, name=Font_name_english, size=Font_size_english)
legend_dup_split.alignment = alignment_center_wrap
legend_dup_split.border = Border(
    left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin")
)

# Add legend for - לא בוצעה אנליזה
legend_dup_split = ws.cell(row=len(df_sorted)+3, column=start_col)
legend_dup_split.value = " - לא בוצעה אנליזה"
legend_dup_split.font = Font(bold=True, name=Font_name_hebrew, size=Font_size_hebrew)
legend_dup_split.alignment = alignment_center_wrap
legend_dup_split.border = Border(
    left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin")
)

# Add legend for ' -- לא קיים ערך סף
legend_dup_split = ws.cell(row=len(df_sorted)+4, column=start_col)
legend_dup_split.value = " -- לא קיים ערך סף"
legend_dup_split.font = Font(bold=True, name=Font_name_hebrew, size=Font_size_hebrew)
legend_dup_split.alignment = alignment_center_wrap
legend_dup_split.border = Border(
    left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin")
)

# Add legend for "חורג מערך VSL"
legend_vsl_cell = ws.cell(row=len(df_sorted)+5 , column=start_col)
legend_vsl_cell.value = "חורג מערך VSL"
legend_vsl_cell.font = Font(bold=True, name=Font_name_hebrew, size=Font_size_hebrew)
legend_vsl_cell.alignment = alignment_center_wrap
legend_vsl_cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
legend_vsl_cell.border = Border(
    left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin")
)

# Add legend for "חורג מערך TIER 1"
legend_tier1_cell = ws.cell(row=len(df_sorted)+6 , column=start_col)
legend_tier1_cell.value = "חורג מערך TIER 1"
legend_tier1_cell.font = Font(bold=True, name=Font_name_hebrew, size=Font_size_hebrew)
legend_tier1_cell.alignment = alignment_center_wrap
legend_tier1_cell.fill = PatternFill(start_color="F7C7AC", end_color="F7C7AC", fill_type="solid")
legend_tier1_cell.border = Border(
    left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin")
)

# Add legend for "ערך גילוי גדול מערך סף"
legend_gray_cell = ws.cell(row=len(df_sorted)+7, column=start_col)
legend_gray_cell.value = "ערך סף גילוי מעבדה גדול מערך סף VSL"
legend_gray_cell.font = Font(bold=True, name=Font_name_hebrew, size=Font_size_hebrew)
legend_gray_cell.alignment = alignment_center_wrap
legend_gray_cell.fill = gray_fill
legend_gray_cell.border = Border(
    left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin")
)

# Save the final Excel file with the provided name
file_path = f'{file_name}.xlsx'
wb.save(file_path)

# Download the Excel file
files.download(file_path)


/tmp/ipython-input-2129743695.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  results_data_TPH = results_data_TPH.applymap(remove_parentheses)
/tmp/ipython-input-2129743695.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  results_data_VOC = results_data_VOC.applymap(remove_parentheses)
/tmp/ipython-input-2129743695.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  results_data_ICP = results_data_ICP.applymap(remove_parentheses)
/tmp/ipython-input-2129743695.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_to_sort['name_prefix'] = rows_to_sort['name'].str.extract(r'([^-\d]+)', expand=False)
/tmp/ipython-input-2129

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>